# Creating a simple cell optimisation

This notebook will explain how to set up an optimisation of simple single compartmental cell with two free parameters that need to be optimised.
As this optimisation is for example purpose only, no real experimental data is used in this notebook.

First we need to import the module that contains all the functionality to create electrical cell models

In [ ]:
import bluepyopt as bpop
import bluepyopt.ephys as ephys
from neuron import h

Load additional modules

In [ ]:
import sys
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

Setting up a cell template
-------------------------
First a template that will describe the cell has to be defined. A template consists of:
* a morphology
* model mechanisms
* model parameters

### Creating a morphology
A morphology can be loaded from a file (SWC or ASC).

In [ ]:
morph = ephys.morphologies.NrnFileMorphology('simple.swc')

By default a Neuron morphology has the following sectionlists: somatic, axonal, apical and basal. Let's create an object that points to the somatic sectionlist. This object will be used later to specify where mechanisms have to be added etc.

In [ ]:
somatic_loc = ephys.locations.NrnSeclistLocation('somatic', seclist_name='somatic')

### Creating a mechanism

Now we can add ion channels to this morphology. Let's add the default Neuron Hodgkin-Huxley mechanism to the soma. 

In [ ]:
hh_mech = ephys.mechanisms.NrnMODMechanism(
        name='hh',
        suffix='hh',
        locations=[somatic_loc])

The 'name' field can be chosen by the user, this name should be unique. The 'suffix' points to the same field in the NMODL file of the channel. 'locations' specifies which sections the mechanism will be added to.

### Creating parameters

Next we need to specify the parameters of the model. A parameter can be in two states: frozen and not-frozen. When a parameter is frozen it has an exact value, otherwise it only has some bounds but the exact value is not known yet.
Let's define first a parameter that sets the capacitance of the soma to a frozen value

In [ ]:
cm_param = ephys.parameters.NrnSectionParameter(
        name='cm',
        param_name='cm',
        value=1.0,
        locations=[somatic_loc],
        frozen=True)

And parameters that represent the maximal conductance of the sodium and potassium channels. These two parameters will be optimised later.

In [ ]:
gnabar_param = ephys.parameters.NrnSectionParameter(                                    
        name='gnabar_hh',
        param_name='gnabar_hh',
        locations=[somatic_loc],
        bounds=[0.05, 0.2],
        frozen=False)     
gkbar_param = ephys.parameters.NrnSectionParameter(
        name='gkbar_hh',
        param_name='gkbar_hh',
        bounds=[0.01, 0.1],
        locations=[somatic_loc],
        frozen=False)

### Creating the template

To create the cell template, we pass all these objects to the constructor of the template

In [ ]:
simple_cell = ephys.models.CellModel(
        name='simple_cell',
        morph=morph,
        mechs=[hh_mech],
        params=[cm_param, gnabar_param, gkbar_param])  

Now we can print out a description of the cell

In [ ]:
print(simple_cell)

With this cell we can build a cell evaluator.

## Setting up a cell evaluator

To optimise the parameters of the cell we need to create cell evaluator object. 
This object will need to know which protocols to inject, which parameters to optimise, etc.

### Creating the protocols

A protocol consists of a set of stimuli, and a set of responses (i.e. recordings). These responses will later be used to calculate
the score of the parameter values.
Let's create two protocols, two square current pulses at somatic`[0]`(0.5) with different amplitudes.
We first need to create a location object

In [ ]:
soma_loc = ephys.locations.NrnSeclistCompLocation(
        name='soma',
        seclist_name='somatic',
        sec_index=0,
        comp_x=0.5)

and then the stimuli, recordings and protocols. For each protocol we add a recording and a stimulus in the soma.

In [ ]:
sweep_protocols = []
for protocol_name, amplitude in [('step1', 0.01), ('step2', 0.05)]:
    stim = ephys.stimuli.NrnSquarePulse(
                step_amplitude=amplitude,
                step_delay=100,
                step_duration=50,
                location=soma_loc,
                total_duration=200)
    rec = ephys.recordings.CompRecording(
            name=f'{protocol_name}.soma.v',
            location=soma_loc,
            variable='v')
    protocol = ephys.protocols.SweepProtocol(protocol_name, [stim], [rec])
    sweep_protocols.append(protocol)
twostep_protocol = ephys.protocols.SequenceProtocol('twostep', protocols=sweep_protocols)

### Running a protocol on a cell

Now we're at a stage where we can actually run a protocol on the cell. We first need to create a Simulator object.

In [ ]:
nrn = ephys.simulators.NrnSimulator()

Define a function for plotting the voltage responses to the stimulim

In [ ]:
def plot_responses(*args):
    fig,ax = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(7,4))
    cmap = plt.get_cmap('viridis', len(args))
    for n,responses in enumerate(args):
        for i,a in enumerate(ax):
            if len(args) == 1:
                col = 'k'
            else:
                col = cmap(n)
            plt.subplot(2,1,i+1)
            key = f'step{i+1}.soma.v'
            a.plot(responses[key]['time'], responses[key]['voltage'], color=col, lw=1)
            a.set_title(f'Step {i+1}')
            a.set_ylim([-90,50])
            a.set_yticks(np.r_[-80 : 60 : 20])
            for side in 'right','top':
                a.spines[side].set_visible(False)
            a.grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
            a.set_ylabel('Vm (mV)')
    ax[-1].set_xlabel('Time (ms)')
    fig.tight_layout()

The run() method of a protocol accepts a cell model, a set of parameter values and a simulator

In [ ]:
default_params = {'gnabar_hh': 0.15, 'gkbar_hh': 0.03}
responses = twostep_protocol.run(cell_model=simple_cell, param_values=default_params, sim=nrn)

Plotting the response traces is now easy:

In [ ]:
plot_responses(responses)

As you can see, when we use different parameter values, the response looks different.

In [ ]:
other_params = {'gnabar_hh': 0.05, 'gkbar_hh': 0.05}
plot_responses(twostep_protocol.run(cell_model=simple_cell, param_values=other_params, sim=nrn))

### Defining eFeatures and objectives

For every response we need to define a set of eFeatures we will use for the fitness calculation later. We have to combine features together into objectives that will be used by the optimalisation algorithm. In this case we will create one objective per feature:

In [ ]:
efel_feature_means = {'step1': {'Spikecount': 1}, 'step2': {'Spikecount': 5}}
efel_feature_stds = {'step1': {'Spikecount': 0.05}, 'step2': {'Spikecount': 0.025}}

objectives = []

for protocol in sweep_protocols:
    stim_start = protocol.stimuli[0].step_delay
    stim_end = stim_start + protocol.stimuli[0].step_duration
    for efel_feature_name, mean in efel_feature_means[protocol.name].items():
        feature_name = '%s.%s' % (protocol.name, efel_feature_name)
        feature = ephys.efeatures.eFELFeature(
                    feature_name,
                    efel_feature_name=efel_feature_name,
                    recording_names={'': '%s.soma.v' % protocol.name},
                    stim_start=stim_start,
                    stim_end=stim_end,
                    exp_mean=mean,
                    exp_std=efel_feature_stds[protocol.name][efel_feature_name])
        objective = ephys.objectives.SingletonObjective(
            feature_name,
            feature)
        objectives.append(objective)

### Creating the cell evaluator

We will need an object that can use these objective definitions to calculate the scores from a protocol response. This is called a ScoreCalculator.

In [ ]:
score_calc = ephys.objectivescalculators.ObjectivesCalculator(objectives) 

Combining everything together we have a CellEvaluator. The CellEvaluator constructor has a field 'parameter_names' which contains the (ordered) list of names of the parameters that are used as input (and will be fitted later on).

In [ ]:
cell_evaluator = ephys.evaluators.CellEvaluator(
        cell_model=simple_cell,
        param_names=['gnabar_hh', 'gkbar_hh'],
        fitness_protocols={twostep_protocol.name: twostep_protocol},
        fitness_calculator=score_calc,
        sim=nrn)

### Evaluating the cell

The cell can now be evaluate for a certain set of parameter values.

In [ ]:
print(cell_evaluator.evaluate_with_dicts(default_params))

## Setting up and running an optimisation

Now that we have a cell template and an evaluator for this cell, we can set up an optimisation.

In [ ]:
offspring_size = 10
optimisation = bpop.optimisations.DEAPOptimisation(
        evaluator=cell_evaluator,
        offspring_size = offspring_size)

And this optimisation can be run for a certain number of generations

In [ ]:
final_pop, hall_of_fame, logs, hist = optimisation.run(max_ngen=5)

## Looking at the optimization results

The optimisation has returned us 4 objects: final population, hall of fame, statistical logs and history. 

The final population contains a list of tuples, with each tuple representing the two parameters of the model

In [ ]:
pop_size = len(final_pop)

print('Final population:\n')
sys.stdout.write('    ')
for param_name in cell_evaluator.param_names:
    sys.stdout.write(f'{param_name:>12s}')
sys.stdout.write('\n')
for i,individual in enumerate(final_pop):
    sys.stdout.write(f'[{i+1:2d}]')
    for par in individual:
        sys.stdout.write(f'{par:12.4f}')
    sys.stdout.write('\n')

The best individual found during the optimisation is the first individual of the hall of fame

In [ ]:
best_ind = hall_of_fame[0]
print('Best individual: ', best_ind)
print('Fitness values: ', best_ind.fitness.values)

In [ ]:
initial_responses = []
for i in tqdm(range(offspring_size)):
    individual = cell_evaluator.param_dict(hist.genealogy_history[i+1])
    responses = twostep_protocol.run(cell_model=simple_cell, param_values=individual, sim=nrn)
    initial_responses.append(responses)

In [ ]:
hof_responses = []
for ind in tqdm(hall_of_fame.items):
    individual = cell_evaluator.param_dict(ind)
    responses = twostep_protocol.run(cell_model=simple_cell, param_values=individual, sim=nrn)
    hof_responses.append(responses)

Let's plot the responses of the initial population...

In [ ]:
plot_responses(*initial_responses)

... and those of the hall of fame of individuals

In [ ]:
plot_responses(*hof_responses)

Let's have a look at the optimisation statistics.
We can plot the minimal score (sum of all objective scores) found in every optimisation. 
The optimisation algorithm uses negative fitness scores, so we actually have to look at the maximum values log.

In [ ]:
gen_numbers = logs.select('gen')
min_fitness = logs.select('min')
max_fitness = logs.select('max')
fig,ax = plt.subplots(1, 1, figsize=(6,4))
ax.plot(gen_numbers, min_fitness, 'k', lw=2, label='Min. fitness')
ax.set_xlabel('Generation #')
ax.set_ylabel('Score (# std)')
ax.legend(loc='best', frameon=False)
ax.set_xlim(min(gen_numbers) - 1, max(gen_numbers) + 1) 
ax.set_ylim(0.9*min(min_fitness), 1.1 * max(min_fitness))
for side in 'right','top':
    ax.spines[side].set_visible(False)
fig.tight_layout()